In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!ls /content/drive/MyDrive/Colab\ Notebooks/ftp

task2  task3


In [ ]:
!pip3 install transformers
!pip3 install tensorflow
!pip3 install numpy
!pip3 install scikit-learn

In [4]:
import xml.etree.ElementTree as ET
import glob
import re

def get_xml_tag_contents(xml_str, tag):
    open_tag = f"<{tag}>"
    close_tag = f"</{tag}>"
    open_tag_end = xml_str.find(open_tag)+len(open_tag)
    close_tag_start = xml_str.find(close_tag, open_tag_end)
    return xml_str[open_tag_end:close_tag_start], xml_str[close_tag_start+len(close_tag):]

def parse_subject_xml(filename):
    with open(filename, "r+") as f:
        xml_str = f.read()
    xml_str, _ = get_xml_tag_contents(xml_str, "INDIVIDUAL")
    id_str, xml_str = get_xml_tag_contents(xml_str, "ID")
    subjectId = id_str.strip()
    writings = []
    while (xml_str.strip() != ""):
        writing = {}
        writing_str, xml_str = get_xml_tag_contents(xml_str, "WRITING")
        
        title_str, writing_str = get_xml_tag_contents(writing_str, "TITLE")
        writing["TITLE"] = title_str.strip()
        date_str, writing_str = get_xml_tag_contents(writing_str, "DATE")
        writing["DATE"] = date_str.strip()
        info_str, writing_str = get_xml_tag_contents(writing_str, "INFO")
        writing["INFO"] = info_str.strip()
        text_str, writing_str = get_xml_tag_contents(writing_str, "TEXT")
        writing["TEXT"] = text_str.strip()

        writings.append(writing)
    return subjectId, writings


training_users = {}
# Add training subjects & posts
filepath = f"/content/drive/MyDrive/Colab Notebooks/ftp/task3/training/*/subject*.xml"
filenames = glob.glob(filepath)
for filename in filenames:
    subjectId, writings = parse_subject_xml(filename)
    if subjectId == "" or writings == []:
        continue
    year = re.findall("/content/drive/MyDrive/Colab\ Notebooks/ftp/task3/training/([0-9]{4})/subject[0-9]*.xml", filename)[0]
    subjectId = f"{year}-{subjectId}"
    training_users[subjectId] = {"WRITINGS": writings}
# Add training subject responses
filepath = f"/content/drive/MyDrive/Colab Notebooks/ftp/task3/training/*/Depression Questionnaires_anon.txt"
filenames = glob.glob(filepath)
for filename in filenames:
    year = re.findall("/content/drive/MyDrive/Colab Notebooks/ftp/task3/training/([0-9]{4})/Depression Questionnaires_anon.txt", filename)[0]
    with open(filename, "r+") as f:
        user_responses = [line.strip().split("\t") for line in f.readlines() if line.strip != ""]
    for user_response in user_responses:
        subjectId = f"{year}-{user_response[0]}"
        responses = user_response[1:]
        training_users[subjectId]['RESPONSES'] = responses
print(f"Loaded {len(training_users)} training users")

# Add testing subjects & posts
testing_users = {}
filepath = f"/content/drive/MyDrive/Colab Notebooks/ftp/task3/data/2021/erisk2021-T3_Subject*.xml"
filenames = glob.glob(filepath)
for filename in filenames:
    subjectId, writings = parse_subject_xml(filename)
    if subjectId == "" or writings == []:
        continue
    testing_users[subjectId] = {"WRITINGS": writings, "RESPONSES": []}
print(f"Loaded {len(testing_users)} testing users")

# TODO: make a model & add predictions to test users




# Output test subject responses
filepath = f"/content/drive/MyDrive/Colab Notebooks/ftp/task3/results/Depression Questionnaires_anon.txt"
with open(filepath, "w+") as f:
    f.write('\n'.join([user + ' ' + ' '.join(testing_users[user]['RESPONSES']) for user in testing_users]))

Loaded 90 training users
Loaded 80 testing users


In [5]:
import numpy as np
import nltk
from nltk.tokenize import word_tokenize  
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import *
nltk.download('words')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
words = set(nltk.corpus.words.words())
text = []
responses=[]
t=""
for key in training_users:
  
  key_response = []
  for i, val in enumerate(training_users[key]["RESPONSES"]):
    if val == '0':
      if i!=15 and i!=17:
        one_hot = [1,0,0,0]
      else:
        one_hot = [1,0,0,0,0,0,0]
    elif val =='1':
        one_hot = [0,1,0,0]
    elif val == '2':
        one_hot = [0,0,1,0]
    elif val == '3':
        one_hot = [0,0,0,1]
    elif val == '1a':
        one_hot = [0,1,0,0,0,0,0]
    elif val =='1b':
        one_hot = [0,0,1,0,0,0,0]
    elif val =='2a':
        one_hot = [0,0,0,1,0,0,0]
    elif val =='2b':
        one_hot = [0,0,0,0,1,0,0]
    elif val =='3a':
        one_hot = [0,0,0,0,0,1,0]
    elif val =='3b':
        one_hot = [0,0,0,0,0,0,1]

    key_response.append(np.asarray(one_hot))

  for dic in training_users[key]["WRITINGS"]:
    no_url = re.sub(r'http\S+', '', dic["TITLE"]+dic["TEXT"])
    phrase = re.sub(r"won\'t", "not", no_url)
    phrase = re.sub(r"can\'t", "not", phrase)
    phrase = re.sub(r"isn\'t", "not", phrase)
    phrase = re.sub(r"aren\'t", "not", phrase)
    phrase = re.sub(r"don\'t", "not", phrase)
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    no_nonascill = re.sub(r'[^\x00-\x7F]+',' ', phrase)
    lowercase = no_nonascill.lower()
    clean = re.sub('\W+',' ', lowercase )
    p_stemmer = PorterStemmer()
    nltk_tokenList = word_tokenize(clean)
    #Stemming
    nltk_stemedList = []
    for word in nltk_tokenList:
        nltk_stemedList.append(p_stemmer.stem(word))
        #nltk_stemedList.append(s_stemmer.stem(word))
    
    #Lemmatization
    wordnet_lemmatizer = WordNetLemmatizer()
    nltk_lemmaList = []
    for word in nltk_stemedList:
        nltk_lemmaList.append(wordnet_lemmatizer.lemmatize(word))

    #Filter stopword
    filtered_sentence = []  
    nltk_stop_words = set(stopwords.words("english"))
    for w in nltk_lemmaList:  
        if w not in nltk_stop_words:  
            filtered_sentence.append(w)  

    #Removing Punctuation
    punctuations="?:!.,;"
    for word in filtered_sentence:
        if word in punctuations:
            filtered_sentence.remove(word)
    
    t = ""
    for word in filtered_sentence:
      if t == "":
        t = t + word
      else:
        t = t + " " + word
    text.append(t)
    responses.append(key_response)


[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [6]:
from transformers import TFBertModel,  BertConfig, BertTokenizerFast
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

In [7]:
nparr=[[]]
for i in range(len(text)):

  arr = np.asarray([[text[i]]])
  arr = np.append( arr, [responses[i]])
  nparr.append(arr)
nparr
df = pd.DataFrame(nparr, columns=['Text', 'Q1','Q2','Q3','Q4','Q5','Q6','Q7','Q8','Q9','Q10','Q11','Q12','Q13','Q14','Q15','Q16','Q17','Q18','Q19','Q20','Q21'])
df = df.drop(df.index[0])
df

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


,Text,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,Q11,Q12,Q13,Q14,Q15,Q16,Q17,Q18,Q19,Q20,Q21
1,im sorri youv neg experi im support ever need ...,"[0, 1, 0, 0]","[0, 0, 0, 1]","[0, 0, 0, 1]","[0, 0, 1, 0]","[0, 0, 0, 1]","[0, 0, 1, 0]","[0, 0, 1, 0]","[0, 0, 0, 1]","[0, 0, 1, 0]","[0, 0, 1, 0]","[0, 1, 0, 0]","[0, 1, 0, 0]","[0, 0, 1, 0]","[0, 0, 0, 1]","[0, 1, 0, 0]","[0, 0, 0, 0, 1, 0, 0]","[0, 1, 0, 0]","[1, 0, 0, 0, 0, 0, 0]","[0, 0, 1, 0]","[0, 0, 1, 0]","[0, 1, 0, 0]"
2,scratch top wrist,"[0, 1, 0, 0]","[0, 0, 0, 1]","[0, 0, 0, 1]","[0, 0, 1, 0]","[0, 0, 0, 1]","[0, 0, 1, 0]","[0, 0, 1, 0]","[0, 0, 0, 1]","[0, 0, 1, 0]","[0, 0, 1, 0]","[0, 1, 0, 0]","[0, 1, 0, 0]","[0, 0, 1, 0]","[0, 0, 0, 1]","[0, 1, 0, 0]","[0, 0, 0, 0, 1, 0, 0]","[0, 1, 0, 0]","[1, 0, 0, 0, 0, 0, 0]","[0, 0, 1, 0]","[0, 0, 1, 0]","[0, 1, 0, 0]"
3,opinion high school anxietyi suffer exam cente...,"[0, 1, 0, 0]","[0, 0, 0, 1]","[0, 0, 0, 1]","[0, 0, 1, 0]","[0, 0, 0, 1]","[0, 0, 1, 0]","[0, 0, 1, 0]","[0, 0, 0, 1]","[0, 0, 1, 0]","[0, 0, 1, 0]","[0, 1, 0, 0]","[0, 1, 0, 0]","[0, 0, 1, 0]","[0, 0, 0, 1]","[0, 1, 0, 0]","[0, 0, 0, 0, 1, 0, 0]","[0, 1, 0, 0]","[1, 0, 0, 0, 0, 0, 0]","[0, 0, 1, 0]","[0, 0, 1, 0]","[0, 1, 0, 0]"
4,im sorri hear wish could give warm comfort hug...,"[0, 1, 0, 0]","[0, 0, 0, 1]","[0, 0, 0, 1]","[0, 0, 1, 0]","[0, 0, 0, 1]","[0, 0, 1, 0]","[0, 0, 1, 0]","[0, 0, 0, 1]","[0, 0, 1, 0]","[0, 0, 1, 0]","[0, 1, 0, 0]","[0, 1, 0, 0]","[0, 0, 1, 0]","[0, 0, 0, 1]","[0, 1, 0, 0]","[0, 0, 0, 0, 1, 0, 0]","[0, 1, 0, 0]","[1, 0, 0, 0, 0, 0, 0]","[0, 0, 1, 0]","[0, 0, 1, 0]","[0, 1, 0, 0]"
5,hi im head bed plea feel free spill thought ch...,"[0, 1, 0, 0]","[0, 0, 0, 1]","[0, 0, 0, 1]","[0, 0, 1, 0]","[0, 0, 0, 1]","[0, 0, 1, 0]","[0, 0, 1, 0]","[0, 0, 0, 1]","[0, 0, 1, 0]","[0, 0, 1, 0]","[0, 1, 0, 0]","[0, 1, 0, 0]","[0, 0, 1, 0]","[0, 0, 0, 1]","[0, 1, 0, 0]","[0, 0, 0, 0, 1, 0, 0]","[0, 1, 0, 0]","[1, 0, 0, 0, 0, 0, 0]","[0, 0, 1, 0]","[0, 0, 1, 0]","[0, 1, 0, 0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46499,remu romulu learn latin class far understood m...,"[0, 1, 0, 0]","[1, 0, 0, 0]","[0, 1, 0, 0]","[1, 0, 0, 0]","[0, 1, 0, 0]","[1, 0, 0, 0]","[1, 0, 0, 0]","[1, 0, 0, 0]","[1, 0, 0, 0]","[0, 1, 0, 0]","[1, 0, 0, 0]","[0, 1, 0, 0]","[1, 0, 0, 0]","[1, 0, 0, 0]","[0, 1, 0, 0]","[0, 1, 0, 0, 0, 0, 0]","[1, 0, 0, 0]","[0, 0, 1, 0, 0, 0, 0]","[1, 0, 0, 0]","[0, 1, 0, 0]","[1, 0, 0, 0]"
46500,would dismiss belgium quit soon forget footbal...,"[0, 1, 0, 0]","[1, 0, 0, 0]","[0, 1, 0, 0]","[1, 0, 0, 0]","[0, 1, 0, 0]","[1, 0, 0, 0]","[1, 0, 0, 0]","[1, 0, 0, 0]","[1, 0, 0, 0]","[0, 1, 0, 0]","[1, 0, 0, 0]","[0, 1, 0, 0]","[1, 0, 0, 0]","[1, 0, 0, 0]","[0, 1, 0, 0]","[0, 1, 0, 0, 0, 0, 0]","[1, 0, 0, 0]","[0, 0, 1, 0, 0, 0, 0]","[1, 0, 0, 0]","[0, 1, 0, 0]","[1, 0, 0, 0]"
46501,fair think would kill ourselv need outsid inte...,"[0, 1, 0, 0]","[1, 0, 0, 0]","[0, 1, 0, 0]","[1, 0, 0, 0]","[0, 1, 0, 0]","[1, 0, 0, 0]","[1, 0, 0, 0]","[1, 0, 0, 0]","[1, 0, 0, 0]","[0, 1, 0, 0]","[1, 0, 0, 0]","[0, 1, 0, 0]","[1, 0, 0, 0]","[1, 0, 0, 0]","[0, 1, 0, 0]","[0, 1, 0, 0, 0, 0, 0]","[1, 0, 0, 0]","[0, 0, 1, 0, 0, 0, 0]","[1, 0, 0, 0]","[0, 1, 0, 0]","[1, 0, 0, 0]"
46502,veri true soon go flander notic differ differ ...,"[0, 1, 0, 0]","[1, 0, 0, 0]","[0, 1, 0, 0]","[1, 0, 0, 0]","[0, 1, 0, 0]","[1, 0, 0, 0]","[1, 0, 0, 0]","[1, 0, 0, 0]","[1, 0, 0, 0]","[0, 1, 0, 0]","[1, 0, 0, 0]","[0, 1, 0, 0]","[1, 0, 0, 0]","[1, 0, 0, 0]","[0, 1, 0, 0]","[0, 1, 0, 0, 0, 0, 0]","[1, 0, 0, 0]","[0, 0, 1, 0, 0, 0, 0]","[1, 0, 0, 0]","[0, 1, 0, 0]","[1, 0, 0, 0]"


In [8]:

df, df_test = train_test_split(df, train_size = 38146, shuffle=False)
df

,Text,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,Q11,Q12,Q13,Q14,Q15,Q16,Q17,Q18,Q19,Q20,Q21
1,im sorri youv neg experi im support ever need ...,"[0, 1, 0, 0]","[0, 0, 0, 1]","[0, 0, 0, 1]","[0, 0, 1, 0]","[0, 0, 0, 1]","[0, 0, 1, 0]","[0, 0, 1, 0]","[0, 0, 0, 1]","[0, 0, 1, 0]","[0, 0, 1, 0]","[0, 1, 0, 0]","[0, 1, 0, 0]","[0, 0, 1, 0]","[0, 0, 0, 1]","[0, 1, 0, 0]","[0, 0, 0, 0, 1, 0, 0]","[0, 1, 0, 0]","[1, 0, 0, 0, 0, 0, 0]","[0, 0, 1, 0]","[0, 0, 1, 0]","[0, 1, 0, 0]"
2,scratch top wrist,"[0, 1, 0, 0]","[0, 0, 0, 1]","[0, 0, 0, 1]","[0, 0, 1, 0]","[0, 0, 0, 1]","[0, 0, 1, 0]","[0, 0, 1, 0]","[0, 0, 0, 1]","[0, 0, 1, 0]","[0, 0, 1, 0]","[0, 1, 0, 0]","[0, 1, 0, 0]","[0, 0, 1, 0]","[0, 0, 0, 1]","[0, 1, 0, 0]","[0, 0, 0, 0, 1, 0, 0]","[0, 1, 0, 0]","[1, 0, 0, 0, 0, 0, 0]","[0, 0, 1, 0]","[0, 0, 1, 0]","[0, 1, 0, 0]"
3,opinion high school anxietyi suffer exam cente...,"[0, 1, 0, 0]","[0, 0, 0, 1]","[0, 0, 0, 1]","[0, 0, 1, 0]","[0, 0, 0, 1]","[0, 0, 1, 0]","[0, 0, 1, 0]","[0, 0, 0, 1]","[0, 0, 1, 0]","[0, 0, 1, 0]","[0, 1, 0, 0]","[0, 1, 0, 0]","[0, 0, 1, 0]","[0, 0, 0, 1]","[0, 1, 0, 0]","[0, 0, 0, 0, 1, 0, 0]","[0, 1, 0, 0]","[1, 0, 0, 0, 0, 0, 0]","[0, 0, 1, 0]","[0, 0, 1, 0]","[0, 1, 0, 0]"
4,im sorri hear wish could give warm comfort hug...,"[0, 1, 0, 0]","[0, 0, 0, 1]","[0, 0, 0, 1]","[0, 0, 1, 0]","[0, 0, 0, 1]","[0, 0, 1, 0]","[0, 0, 1, 0]","[0, 0, 0, 1]","[0, 0, 1, 0]","[0, 0, 1, 0]","[0, 1, 0, 0]","[0, 1, 0, 0]","[0, 0, 1, 0]","[0, 0, 0, 1]","[0, 1, 0, 0]","[0, 0, 0, 0, 1, 0, 0]","[0, 1, 0, 0]","[1, 0, 0, 0, 0, 0, 0]","[0, 0, 1, 0]","[0, 0, 1, 0]","[0, 1, 0, 0]"
5,hi im head bed plea feel free spill thought ch...,"[0, 1, 0, 0]","[0, 0, 0, 1]","[0, 0, 0, 1]","[0, 0, 1, 0]","[0, 0, 0, 1]","[0, 0, 1, 0]","[0, 0, 1, 0]","[0, 0, 0, 1]","[0, 0, 1, 0]","[0, 0, 1, 0]","[0, 1, 0, 0]","[0, 1, 0, 0]","[0, 0, 1, 0]","[0, 0, 0, 1]","[0, 1, 0, 0]","[0, 0, 0, 0, 1, 0, 0]","[0, 1, 0, 0]","[1, 0, 0, 0, 0, 0, 0]","[0, 0, 1, 0]","[0, 0, 1, 0]","[0, 1, 0, 0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38142,beauti soft calm,"[1, 0, 0, 0]","[1, 0, 0, 0]","[1, 0, 0, 0]","[1, 0, 0, 0]","[1, 0, 0, 0]","[1, 0, 0, 0]","[1, 0, 0, 0]","[1, 0, 0, 0]","[1, 0, 0, 0]","[1, 0, 0, 0]","[1, 0, 0, 0]","[1, 0, 0, 0]","[1, 0, 0, 0]","[1, 0, 0, 0]","[0, 1, 0, 0]","[0, 1, 0, 0, 0, 0, 0]","[1, 0, 0, 0]","[1, 0, 0, 0, 0, 0, 0]","[0, 1, 0, 0]","[0, 1, 0, 0]","[1, 0, 0, 0]"
38143,found idea sinc wear nail polish pretti much t...,"[1, 0, 0, 0]","[1, 0, 0, 0]","[1, 0, 0, 0]","[1, 0, 0, 0]","[1, 0, 0, 0]","[1, 0, 0, 0]","[1, 0, 0, 0]","[1, 0, 0, 0]","[1, 0, 0, 0]","[1, 0, 0, 0]","[1, 0, 0, 0]","[1, 0, 0, 0]","[1, 0, 0, 0]","[1, 0, 0, 0]","[0, 1, 0, 0]","[0, 1, 0, 0, 0, 0, 0]","[1, 0, 0, 0]","[1, 0, 0, 0, 0, 0, 0]","[0, 1, 0, 0]","[0, 1, 0, 0]","[1, 0, 0, 0]"
38144,start bullet journal three week ago lot inspir...,"[1, 0, 0, 0]","[1, 0, 0, 0]","[1, 0, 0, 0]","[1, 0, 0, 0]","[1, 0, 0, 0]","[1, 0, 0, 0]","[1, 0, 0, 0]","[1, 0, 0, 0]","[1, 0, 0, 0]","[1, 0, 0, 0]","[1, 0, 0, 0]","[1, 0, 0, 0]","[1, 0, 0, 0]","[1, 0, 0, 0]","[0, 1, 0, 0]","[0, 1, 0, 0, 0, 0, 0]","[1, 0, 0, 0]","[1, 0, 0, 0, 0, 0, 0]","[0, 1, 0, 0]","[0, 1, 0, 0]","[1, 0, 0, 0]"
38145,okay like thursday either,"[1, 0, 0, 0]","[1, 0, 0, 0]","[1, 0, 0, 0]","[1, 0, 0, 0]","[1, 0, 0, 0]","[1, 0, 0, 0]","[1, 0, 0, 0]","[1, 0, 0, 0]","[1, 0, 0, 0]","[1, 0, 0, 0]","[1, 0, 0, 0]","[1, 0, 0, 0]","[1, 0, 0, 0]","[1, 0, 0, 0]","[0, 1, 0, 0]","[0, 1, 0, 0, 0, 0, 0]","[1, 0, 0, 0]","[1, 0, 0, 0, 0, 0, 0]","[0, 1, 0, 0]","[0, 1, 0, 0]","[1, 0, 0, 0]"


In [9]:
Q1=[]
Q2=[]
Q3=[]
Q4=[]
Q5=[]
Q6=[]
Q7=[]
Q8=[]
Q9=[]
Q10=[]
Q11=[]
Q12=[]
Q13=[]
Q14=[]
Q15=[]
Q16=[]
Q17=[]
Q18=[]
Q19=[]
Q20=[]
Q21=[]
for i in df["Q1"]:
  Q1.append(i)
for i in df["Q2"]:
  Q2.append(i)
for i in df["Q3"]:
  Q3.append(i)
for i in df["Q4"]:
  Q4.append(i)
for i in df["Q5"]:
  Q5.append(i)
for i in df["Q6"]:
  Q6.append(i)
for i in df["Q7"]:
  Q7.append(i)
for i in df["Q8"]:
  Q8.append(i)
for i in df["Q9"]:
  Q9.append(i)
for i in df["Q10"]:
  Q10.append(i)
for i in df["Q11"]:
  Q11.append(i)
for i in df["Q12"]:
  Q12.append(i)
for i in df["Q13"]:
  Q13.append(i)
for i in df["Q14"]:
  Q14.append(i)
for i in df["Q15"]:
  Q15.append(i)
for i in df["Q16"]:
  Q16.append(i)
for i in df["Q17"]:
  Q17.append(i)
for i in df["Q18"]:
  Q18.append(i)
for i in df["Q19"]:
  Q19.append(i)
for i in df["Q20"]:
  Q20.append(i)
for i in df["Q21"]:
  Q21.append(i)
Q1 = np.asarray(Q1)
Q2 = np.asarray(Q2)
Q3 = np.asarray(Q3)
Q4 = np.asarray(Q4)
Q5 = np.asarray(Q5)
Q6 = np.asarray(Q6)
Q7 = np.asarray(Q7)
Q8 = np.asarray(Q8)
Q9 = np.asarray(Q9)
Q10 = np.asarray(Q10)
Q11 = np.asarray(Q11)
Q12 = np.asarray(Q12)
Q13 = np.asarray(Q13)
Q14 = np.asarray(Q14)
Q15 = np.asarray(Q15)
Q16 = np.asarray(Q16)
Q17 = np.asarray(Q17)
Q18 = np.asarray(Q18)
Q19 = np.asarray(Q19)
Q20 = np.asarray(Q20)
Q21 = np.asarray(Q21)

In [10]:
Q1_test=[]
Q2_test=[]
Q3_test=[]
Q4_test=[]
Q5_test=[]
Q6_test=[]
Q7_test=[]
Q8_test=[]
Q9_test=[]
Q10_test=[]
Q11_test=[]
Q12_test=[]
Q13_test=[]
Q14_test=[]
Q15_test=[]
Q16_test=[]
Q17_test=[]
Q18_test=[]
Q19_test=[]
Q20_test=[]
Q21_test=[]
for i in df_test["Q1"]:
  Q1_test.append(i)
for i in df_test["Q2"]:
  Q2_test.append(i)
for i in df_test["Q3"]:
  Q3_test.append(i)
for i in df_test["Q4"]:
  Q4_test.append(i)
for i in df_test["Q5"]:
  Q5_test.append(i)
for i in df_test["Q6"]:
  Q6_test.append(i)
for i in df_test["Q7"]:
  Q7_test.append(i)
for i in df_test["Q8"]:
  Q8_test.append(i)
for i in df_test["Q9"]:
  Q9_test.append(i)
for i in df_test["Q10"]:
  Q10_test.append(i)
for i in df_test["Q11"]:
  Q11_test.append(i)
for i in df_test["Q12"]:
  Q12_test.append(i)
for i in df_test["Q13"]:
  Q13_test.append(i)
for i in df_test["Q14"]:
  Q14_test.append(i)
for i in df_test["Q15"]:
  Q15_test.append(i)
for i in df_test["Q16"]:
  Q16_test.append(i)
for i in df_test["Q17"]:
  Q17_test.append(i)
for i in df_test["Q18"]:
  Q18_test.append(i)
for i in df_test["Q19"]:
  Q19_test.append(i)
for i in df_test["Q20"]:
  Q20_test.append(i)
for i in df_test["Q21"]:
  Q21_test.append(i)
Q1_test = np.asarray(Q1_test)
Q2_test = np.asarray(Q2_test)
Q3_test = np.asarray(Q3_test)
Q4_test = np.asarray(Q4_test)
Q5_test = np.asarray(Q5_test)
Q6_test = np.asarray(Q6_test)
Q7_test = np.asarray(Q7_test)
Q8_test = np.asarray(Q8_test)
Q9_test = np.asarray(Q9_test)
Q10_test = np.asarray(Q10_test)
Q11_test = np.asarray(Q11_test)
Q12_test = np.asarray(Q12_test)
Q13_test = np.asarray(Q13_test)
Q14_test = np.asarray(Q14_test)
Q15_test = np.asarray(Q15_test)
Q16_test = np.asarray(Q16_test)
Q17_test = np.asarray(Q17_test)
Q18_test = np.asarray(Q18_test)
Q19_test = np.asarray(Q19_test)
Q20_test = np.asarray(Q20_test)
Q21_test = np.asarray(Q21_test)

In [11]:



# for i in range(1,22):
#     df['Q'+str(i)+'_label'] = pd.Categorical(df['Q'+str(i)])
#     df['Q'+str(i)] = df['Q'+str(i)+'_label'].cat.codes

model_name = 'bert-base-uncased'
max_length = 128

config = BertConfig.from_pretrained(model_name)
config.output_hidden_states = False

tokenizer = BertTokenizerFast.from_pretrained(pretrained_model_name_or_path = model_name, config = config)

transformer_model = TFBertModel.from_pretrained(model_name, config = config)

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [12]:
bert = transformer_model.layers[0]


input_ids = Input(shape=(max_length,), name='input_ids', dtype='int32')
inputs = {'input_ids': input_ids}

bert_model = bert(inputs)[1]
dropout = Dropout(config.hidden_dropout_prob, name='pooled_output')
pooled_output = dropout(bert_model, training=False)

q1 = Dense(units=4, kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='q1', activation='softmax')(pooled_output)
q2 = Dense(units=4, kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='q2', activation='softmax')(pooled_output)
q3 = Dense(units=4, kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='q3', activation='softmax')(pooled_output)
q4 = Dense(units=4, kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='q4', activation='softmax')(pooled_output)
q5 = Dense(units=4, kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='q5', activation='softmax')(pooled_output)
q6 = Dense(units=4, kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='q6', activation='softmax')(pooled_output)
q7 = Dense(units=4, kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='q7', activation='softmax')(pooled_output)
q8 = Dense(units=4, kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='q8', activation='softmax')(pooled_output)
q9 = Dense(units=4, kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='q9', activation='softmax')(pooled_output)
q10 = Dense(units=4, kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='q10', activation='softmax')(pooled_output)
q11 = Dense(units=4, kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='q11', activation='softmax')(pooled_output)
q12 = Dense(units=4, kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='q12', activation='softmax')(pooled_output)
q13 = Dense(units=4, kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='q13', activation='softmax')(pooled_output)
q14 = Dense(units=4, kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='q14', activation='softmax')(pooled_output)
q15 = Dense(units=4, kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='q15', activation='softmax')(pooled_output)
q16 = Dense(units=7, kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='q16', activation='softmax')(pooled_output)
q17 = Dense(units=4, kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='q17', activation='softmax')(pooled_output)
q18 = Dense(units=7, kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='q18', activation='softmax')(pooled_output)
q19 = Dense(units=4, kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='q19', activation='softmax')(pooled_output)
q20 = Dense(units=4, kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='q20', activation='softmax')(pooled_output)
q21 = Dense(units=4, kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='q21', activation='softmax')(pooled_output)


outputs = {'q1': q1,
           'q2': q2,
           'q3': q3,
           'q4': q4,
           'q5': q5,
           'q6': q6,
           'q7': q7,
           'q8': q8,
           'q9': q9,
           'q10': q10,
           'q11': q11,
           'q12': q12,
           'q13': q13,
           'q14': q14,
           'q15': q15,
           'q16': q16,
           'q17': q17,
           'q18': q18,
           'q19': q19,
           'q20': q20,
           'q21': q21}

In [13]:
model = Model(inputs=inputs, outputs=outputs, name='BERT_MultiLabel_MultiClass')
model.summary()

Model: "BERT_MultiLabel_MultiClass"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 128)]        0                                            
__________________________________________________________________________________________________
bert (TFBertMainLayer)          TFBaseModelOutputWit 109482240   input_ids[0][0]                  
__________________________________________________________________________________________________
pooled_output (Dropout)         (None, 768)          0           bert[0][1]                       
__________________________________________________________________________________________________
q1 (Dense)                      (None, 4)            3076        pooled_output[0][0]              
_________________________________________________________________________

In [14]:
# Set an optimizer
optimizer = Adam(
    learning_rate=5e-5,
    epsilon=1e-08,
    decay=0.01,
    clipnorm=1.0)

# Set loss and metrics
loss = {'q1': CategoricalCrossentropy(from_logits = True), 'q2': CategoricalCrossentropy(from_logits = True),
        'q3': CategoricalCrossentropy(from_logits = True), 'q4': CategoricalCrossentropy(from_logits = True),
        'q5': CategoricalCrossentropy(from_logits = True), 'q6': CategoricalCrossentropy(from_logits = True),
        'q7': CategoricalCrossentropy(from_logits = True), 'q8': CategoricalCrossentropy(from_logits = True),
        'q9': CategoricalCrossentropy(from_logits = True), 'q10': CategoricalCrossentropy(from_logits = True),
        'q11': CategoricalCrossentropy(from_logits = True), 'q12': CategoricalCrossentropy(from_logits = True),
        'q13': CategoricalCrossentropy(from_logits = True), 'q14': CategoricalCrossentropy(from_logits = True),
        'q15': CategoricalCrossentropy(from_logits = True), 'q16': CategoricalCrossentropy(from_logits = True),
        'q17': CategoricalCrossentropy(from_logits = True), 'q18': CategoricalCrossentropy(from_logits = True),
        'q19': CategoricalCrossentropy(from_logits = True), 'q20': CategoricalCrossentropy(from_logits = True),
        'q21': CategoricalCrossentropy(from_logits = True)}


metric = {'q1': CategoricalAccuracy('accuracy'), 'q2': CategoricalAccuracy('accuracy'),
        'q3': CategoricalAccuracy('accuracy'), 'q4': CategoricalAccuracy('accuracy'),
        'q5': CategoricalAccuracy('accuracy'), 'q6':CategoricalAccuracy('accuracy'),
        'q7': CategoricalAccuracy('accuracy'), 'q8': CategoricalAccuracy('accuracy'),
        'q9': CategoricalAccuracy('accuracy'), 'q10': CategoricalAccuracy('accuracy'),
        'q11': CategoricalAccuracy('accuracy'), 'q12': CategoricalAccuracy('accuracy'),
        'q13': CategoricalAccuracy('accuracy'), 'q14': CategoricalAccuracy('accuracy'),
        'q15': CategoricalAccuracy('accuracy'), 'q16': CategoricalAccuracy('accuracy'),
        'q17': CategoricalAccuracy('accuracy'), 'q18': CategoricalAccuracy('accuracy'),
        'q19': CategoricalAccuracy('accuracy'), 'q20':CategoricalAccuracy('accuracy'),
        'q21': CategoricalAccuracy('accuracy')}

# Compile the model
model.compile(optimizer = optimizer, loss = loss, metrics = metric)

# Ready output data for the model
y_q1 = Q1
y_q2 = Q2
y_q3 = Q3
y_q4 = Q4
y_q5 = Q5
y_q6 = Q6
y_q7 = Q7
y_q8 = Q8
y_q9 = Q9
y_q10 = Q10
y_q11 = Q11
y_q12 = Q12
y_q13 = Q13
y_q14 = Q14
y_q15 = Q15
y_q16 = Q16
y_q17 = Q17
y_q18 = Q18
y_q19 = Q19
y_q20 = Q20
y_q21 = Q21

#Tokenize the input (takes some time)
x = tokenizer(
    text=df['Text'].to_list(),
    add_special_tokens=True,
    max_length = max_length,
    truncation=True,
    padding = True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = False,
    verbose = True)


In [15]:
history = model.fit(
    x={'input_ids': x['input_ids']},
    y={'q1': y_q1, 'q2': y_q2,
       'q3': y_q3, 'q4': y_q4,
       'q5': y_q5, 'q6': y_q6,
       'q7': y_q7, 'q8': y_q8,
       'q9': y_q9, 'q10': y_q10,
       'q11': y_q11, 'q12': y_q12,
       'q13': y_q13, 'q14': y_q14,
       'q15': y_q15, 'q16': y_q16,
       'q17': y_q17, 'q18': y_q18,
       'q19': y_q19, 'q20': y_q20,
       'q21': y_q21},
    validation_split=0.3,
    batch_size=64,
    epochs=15)

Epoch 1/15


ResourceExhaustedError: ignored

In [ ]:
# Save Model
model.save("/content/drive/MyDrive/pre15epoch_model.tf")

In [ ]:
test_y_q1 = Q1_test
test_y_q2 = Q2_test
test_y_q3 = Q3_test
test_y_q4 = Q4_test
test_y_q5 = Q5_test
test_y_q6 = Q6_test
test_y_q7 = Q7_test
test_y_q8 = Q8_test
test_y_q9 = Q9_test
test_y_q10 = Q10_test
test_y_q11 = Q11_test
test_y_q12 = Q12_test
test_y_q13 = Q13_test
test_y_q14 = Q14_test
test_y_q15 = Q15_test
test_y_q16 = Q16_test
test_y_q17 = Q17_test
test_y_q18 = Q18_test
test_y_q19 = Q19_test
test_y_q20 = Q20_test
test_y_q21 = Q21_test


test_x = tokenizer(
    text = df_test['Text'].to_list(),
    add_special_tokens=True,
    max_length=max_length,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = False,
    verbose = True)

# Run evaluation
model_eval = model.evaluate(
    x={'input_ids': test_x['input_ids']},
    y={'q1': test_y_q1, 'q2': test_y_q2,
       'q3': test_y_q3, 'q4': test_y_q4,
       'q5': test_y_q5, 'q6': test_y_q6,
       'q7': test_y_q7, 'q8': test_y_q8,
       'q9': test_y_q9, 'q10': test_y_q10,
       'q11': test_y_q11, 'q12': test_y_q12,
       'q13': test_y_q13, 'q14': test_y_q14,
       'q15': test_y_q15, 'q16': test_y_q16,
       'q17': test_y_q17, 'q18': test_y_q18,
       'q19': test_y_q19, 'q20': test_y_q20,
       'q21': test_y_q21}
)

In [ ]:
preds = model.predict(test_x["input_ids"])
preds

In [ ]:
keys_test=[]
for i,key in enumerate(training_users):
  if i>=75:
    keys_test.append(key)
test_users = {key: training_users[key] for key in keys_test}

count = 0
y_preds = np.zeros([15,21])
for i,key in enumerate(test_users):

    for k in range(1,22):
      q = preds["q"+str(k)][count : count+len(test_users[key]["WRITINGS"])]
      q = np.asarray(q)
      q = np.mean(q, axis = 0)

      y_preds[i][k-1] = np.argmax(q)
    count += len(test_users[key]["WRITINGS"])

In [ ]:

y_preds = [[str(i) for i in item] for item in y_preds]

for i in range(15):
  for j in range(21):
    val = y_preds[i][j]
    if j!=15 and j!=17:
      if val == '0.0':
        y_preds[i][j] = '0'
      elif val == '1.0':
        y_preds[i][j] = '1'
      elif val == '2.0':
        y_preds[i][j] = '2'
      elif val == '3.0':
        y_preds[i][j]='3'
    else:
      if val == '0.0':
        y_preds[i][j] = '0'
      elif val == '1.0':
        y_preds[i][j] = '1a'
      elif val == '2.0':
        y_preds[i][j] = '1b'
      elif val == '3.0':
        y_preds[i][j]='2a'
      elif val == '4.0':
        y_preds[i][j]='2b'
      elif val == '5.0':
        y_preds[i][j]='3a'
      elif val == '6.0':
        y_preds[i][j]='3b'

correct_count = []
depression_level_predicted =[]
depression_level_truth = []

for i, key in enumerate(test_users):
  arr = test_users[key]["RESPONSES"]
  count=0
  depr_pred=0
  depr_truth=0
  for j in range(21):
    depr_truth = depr_truth + int(arr[j][0])
    depr_pred = depr_pred + int(y_preds[i][j][0])



    if arr[j] == y_preds[i][j]:
      count+=1

  correct_count.append(count)
  depression_level_truth.append(depr_truth)
  depression_level_predicted.append(depr_pred)


true_cat = []
predicted_cat = []
for i in range(15):
  val1 = depression_level_truth[i]
  val2 = depression_level_predicted[i]

  if val1>=0 and val1<=9:
    true_cat.append("minimal")
  elif val1>=10 and val1<=18:
    true_cat.append("mild")
  elif val1>=19 and val1<=29:
    true_cat.append("moderate")
  else:
    true_cat.append("severe")

  if val2>=0 and val2<=9:
    predicted_cat.append("minimal")
  elif val2>=10 and val2<=18:
    predicted_cat.append("mild")
  elif val2>=19 and val2<=29:
    predicted_cat.append("moderate")
  else:
    predicted_cat.append("severe")


  
print(correct_count)
print("average % of correct responses: ", 100*np.mean(np.asarray(correct_count))/21.)

abs_depression = np.absolute(np.array(depression_level_truth) - np.array(depression_level_predicted))
print("absolute difference in deprresion level ranging (0,63) : ",abs_depression)
print("average absolute difference: ", np.mean(abs_depression))


# for i in y_preds:
#   print(i)
# print()  
# for key in test_users:
#   print(test_users[key]["RESPONSES"])


In [ ]:
text = []
responses=[]
t=""
for key in testing_users:
  

  for dic in testing_users[key]["WRITINGS"]:
    t = dic["TITLE"] + dic["TEXT"]
    text.append(t)




In [ ]:
nparr=[[]]
for i in range(len(text)):

  arr = np.asarray([[text[i]]])
  arr = np.append( arr, [])
  nparr.append(arr)
nparr
df = pd.DataFrame(nparr, columns=['Text'])
df = df.drop(df.index[0])
df

In [ ]:
test_x = tokenizer(
    text = df['Text'].to_list(),
    add_special_tokens=True,
    max_length=max_length,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = False,
    verbose = True)


In [ ]:
preds = model.predict(test_x["input_ids"])


In [ ]:
len(preds["q1"])

In [ ]:

count = 0
y_preds = np.zeros([80,21])
for i,key in enumerate(testing_users):

    for k in range(1,22):
      q = preds["q"+str(k)][count : count+len(testing_users[key]["WRITINGS"])]
      q = np.asarray(q)
      q = np.mean(q, axis = 0)

      y_preds[i][k-1] = np.argmax(q)
    count += len(testing_users[key]["WRITINGS"])

In [ ]:
y_preds = [[str(i) for i in item] for item in y_preds]

for i in range(80):
  for j in range(21):
    val = y_preds[i][j]
    if j!=15 and j!=17:
      if val == '0.0':
        y_preds[i][j] = '0'
      elif val == '1.0':
        y_preds[i][j] = '1'
      elif val == '2.0':
        y_preds[i][j] = '2'
      elif val == '3.0':
        y_preds[i][j]='3'
    else:
      if val == '0.0':
        y_preds[i][j] = '0'
      elif val == '1.0':
        y_preds[i][j] = '1a'
      elif val == '2.0':
        y_preds[i][j] = '1b'
      elif val == '3.0':
        y_preds[i][j]='2a'
      elif val == '4.0':
        y_preds[i][j]='2b'
      elif val == '5.0':
        y_preds[i][j]='3a'
      elif val == '6.0':
        y_preds[i][j]='3b'



In [ ]:
len(testing_users)

In [ ]:
i = 0
for key in testing_users:
  testing_users[key]["RESPONSES"] = y_preds[i]
  i=i +1

In [ ]:
filepath = f"/content/drive/MyDrive/Colab Notebooks/ftp/task3/results/Depression Questionnaires_anon.txt"
with open(filepath, "w+") as f:
    f.write('\n'.join([user + ' ' + ' '.join(testing_users[user]['RESPONSES']) for user in testing_users]))